In [1]:
import src
from GUI.handlers import NotebookLogger
from src.base_sweep import BaseSweep
from src.sweep0d import Sweep0D
from src.sweep1d import Sweep1D
from src.sweep2d import Sweep2D
from src.daq_driver import Daq, DaqAOChannel, DaqAIChannel
from src.util import connect_to_station, connect_station_instruments
from src.tracking import *
import nidaqmx
import time, os
import numpy as np
import qcodes as qc
from qcodes import initialise_or_create_database_at, load_by_run_spec
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes.tests.instrument_mocks import DummyInstrument

2020-06-10 17:14:03,901 ¦ qcodes.logger.logger ¦ INFO ¦ logger ¦ start_logger ¦ 234 ¦ QCoDes logger setup completed
2020-06-10 17:14:05,645 ¦ qcodes.logger.logger ¦ INFO ¦ logger ¦ log_qcodes_versions ¦ 278 ¦ QCoDeS version: 0.13.0+123.g54d069ed2
2020-06-10 17:14:05,647 ¦ qcodes.logger.logger ¦ INFO ¦ logger ¦ log_qcodes_versions ¦ 279 ¦ QCoDeS installed in editable mode: True
2020-06-10 17:14:05,649 ¦ qcodes.logger.logger ¦ INFO ¦ logger ¦ log_qcodes_versions ¦ 280 ¦ QCoDeS requirements versions: {'numpy': '1.16.3', 'pyvisa': '1.9.1', 'h5py': '2.9.0', 'websockets': '7.0', 'jsonschema': '3.0.1', 'ruamel.yaml': '0.15.97', 'pyzmq': '19.0.0', 'wrapt': '1.11.1', 'pandas': '0.24.2', 'tabulate': '0.8.7', 'tqdm': '4.32.1', 'opencensus-ext-azure': '1.0.2', 'matplotlib': '3.0.3', 'requirements-parser': '0.2.0', 'dataclasses': '0.7', 'importlib-metadata': '1.6.0', 'pyqtgraph': '0.10.0', 'slacker': 'Not installed', 'coverage': 'Not installed'}
2020-06-10 17:14:06,247 ¦ matplotlib ¦ DEBUG ¦ __init

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Elliott Runburg\.qcodes\logs\command_history.log append
Mode           : backup
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Elliott Runburg\.qcodes\logs\200610-10660-qcodes.log


In [2]:
station = connect_to_station()
devices = connect_station_instruments(station)

2020-06-10 17:14:07,597 ¦ qcodes.configuration.config ¦ DEBUG ¦ config ¦ load_config ¦ 326 ¦ Loading config from C:\Users\Elliott Runburg\qcodesrc.json
2020-06-10 17:14:07,605 ¦ qcodes.configuration.config ¦ DEBUG ¦ config ¦ load_config ¦ 326 ¦ Loading config from C:\Users\Elliott Runburg\MeasureIt\cfg\qcodesrc.json
2020-06-10 17:14:07,637 ¦ qcodes.instrument.base ¦ DEBUG ¦ base ¦ get_idn ¦ 466 ¦ [Dummy1(DummyInstrument)] Error getting or interpreting *IDN?: ''
2020-06-10 17:14:07,648 ¦ qcodes.instrument.base ¦ DEBUG ¦ base ¦ get_idn ¦ 466 ¦ [Dummy2(DummyInstrument)] Error getting or interpreting *IDN?: ''


In [4]:
# initialize the lakeshore

ls = Model_372('lakeshore_372', 'TCPIP::10.155.95.127::7777::SOCKET')
ls.ch01.enabled(False)
ls.ch02.enabled(False)
ls.ch03.enabled(False)
ls.ch05.enabled(False)
ls.ch06.enabled(False)
for ch in ls.channels:
    print(f'Temperature of {ch.short_name} ({"on" if ch.enabled() else "off"}): {ch.temperature()} {ch.units()}')

VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

In [ ]:
# initialize the AMI

from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430
instrument = AMI430("testmagnet", address="10.155.94.127", port=7180, has_current_rating=False)

In [2]:
# initialize the DAQ
daq = Daq("Dev1", "testdaq")

DaqError: Device identifier is invalid.
Device Specified: testdaq

Status Code: -200220

In [3]:
dummy1 = devices['Dummy1']
dummy2 = devices['Dummy2']

In [3]:
daq.ai0.voltage.label="I"
daq.ai1.voltage.label="Vxx"
#daq.ai2.voltage.label="Vxy"
#params = (daq.ai0.voltage)
%matplotlib qt

In [ ]:
s = Sweep1D(daq.ao0.voltage, 4, 2, 0.05, inter_delay = 0.2, bidirectional=True, plot_data=False, save_data=False)

In [5]:
# testing 2d sweep
# Set the sweep parameters -
# [ Parameter, start point, end point, step ]
in_params = [dummy1.dac1, -0.5, 0, 0.01]
out_params = [dummy1.dac2, 1, 2, 0.1]
s2d = Sweep2D(in_params, out_params, inter_delay = 0.2, save_data=False)
s2d.follow_param(dummy1.dac3)

In [8]:
%matplotlib qt
s2d.start()

NameError: name 's2d' is not defined

In [7]:
s2d.kill()


In [4]:
fn = f'{os.environ["MeasureItHome"]}\\cfg\\sweep2d.json'


In [5]:
s2d2 = BaseSweep.init_from_json(fn, station)

In [16]:
s2d2.kill()

In [6]:
%matplotlib qt
s2d2.start()

2020-06-10 17:14:30,446 ¦ matplotlib.pyplot ¦ DEBUG ¦ pyplot ¦ switch_backend ¦ 211 ¦ Loaded backend Qt5Agg version unknown.
2020-06-10 17:14:30,448 ¦ qcodes.dataset.measurements ¦ INFO ¦ measurements ¦ _register_parameter ¦ 963 ¦ Registered Dummy1_dac1 in the Measurement.
2020-06-10 17:14:30,451 ¦ qcodes.dataset.measurements ¦ INFO ¦ measurements ¦ _register_parameter ¦ 963 ¦ Registered time in the Measurement.
2020-06-10 17:14:30,453 ¦ qcodes.dataset.measurements ¦ INFO ¦ measurements ¦ _register_parameter ¦ 963 ¦ Registered Dummy1_dac2 in the Measurement.
2020-06-10 17:14:30,514 ¦ qcodes.dataset.measurements ¦ INFO ¦ measurements ¦ _register_parameter ¦ 963 ¦ Registered Dummy1_dac2 in the Measurement.
2020-06-10 17:14:30,516 ¦ qcodes.dataset.measurements ¦ INFO ¦ measurements ¦ _register_parameter ¦ 963 ¦ Registered Dummy1_dac3 in the Measurement.
2020-06-10 17:14:30,518 ¦ qcodes.dataset.measurements ¦ INFO ¦ measurements ¦ _register_parameter ¦ 963 ¦ Registered Dummy1_dac2 in the M

Sweeping Gate dac2 to 1 V
Ramping Gate dac2 to 1 . . . 


In [ ]:
s = Sweep0D(save_data=False)

In [ ]:
s.follow_param(daq.ai0.voltage)

In [ ]:
s.start()

In [ ]:
s.stop()

In [ ]:
s.end


In [ ]:
s.begin

In [ ]:
from PyQt5.QtCore import QThread
QThread.currentThreadId()